In [1]:
import os
import tensorflow as tf
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from gslTranslater.constants import *
from gslTranslater.utils.common import read_yaml, create_directories

In [2]:
os.chdir("../")

In [3]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    cnn_lstm_model_path: Path
    train_csv: Path
    validate_csv: Path
    test_csv: Path
    data_dir: Path  # Path to the directory where image data is stored
    params_epochs: int
    params_batch_size: int
    params_image_size: list
    max_seq_length: int
    learning_rate: float
    classes: int

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        data_ingestion = self.config.data_ingestion
        params = self.params

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            cnn_lstm_model_path=Path(prepare_base_model.cnn_lstm_model_path),
            train_csv=Path(data_ingestion.train_csv),
            validate_csv=Path(data_ingestion.validate_csv),
            test_csv=Path(data_ingestion.test_csv),
            data_dir=Path(data_ingestion.data_dir),  # Set the data directory
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_image_size=params.IMAGE_SIZE,
            max_seq_length=params.MAX_SEQ_LENGTH,
            learning_rate=params.LEARNING_RATE,
            classes=params.CLASSES
        )

        return training_config

In [15]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = None

    def load_or_process_dataset(self, csv_path, dataset_type, max_seq_length):
        # Define paths for saving the features and labels
        features_save_dir = os.path.join(self.config.root_dir, 'features')
        os.makedirs(features_save_dir, exist_ok=True)
        features_path = os.path.join(features_save_dir, f"{dataset_type}_features.npy")
        labels_path = os.path.join(features_save_dir, f"{dataset_type}_labels.npy")

        # Check if the features and labels already exist
        if os.path.exists(features_path) and os.path.exists(labels_path):
            print(f"Loading existing {dataset_type} features and labels...")
            data = np.load(features_path)
            labels = np.load(labels_path)
            return data, labels

        print(f"Processing {dataset_type} dataset...")

        df = pd.read_csv(csv_path)

        data = []
        labels = []

        for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {dataset_type}", unit="item", ncols=100):
            frames_path = os.path.join(self.config.data_dir, row['Path'].replace('/', '\\'))
            frames = sorted([os.path.join(frames_path, img) for img in os.listdir(frames_path) if img.endswith('.jpg')])

            sequence = []
            for frame in frames:
                image = tf.keras.preprocessing.image.load_img(frame, target_size=tuple(self.config.params_image_size[:-1]))
                image = tf.keras.preprocessing.image.img_to_array(image)
                image = tf.keras.applications.resnet.preprocess_input(image)
                sequence.append(image)

            data.append(sequence)
            labels.append(row['Gloss'])

        # Pad sequences to ensure uniform shape
        data = tf.keras.preprocessing.sequence.pad_sequences(data, maxlen=max_seq_length, padding='post', dtype='float32')
        labels = np.array(labels)

        # Save the features and labels
        np.save(features_path, data)
        np.save(labels_path, labels)

        return data, labels

    def encode_labels(self, labels):
        label_encoder = LabelEncoder()
        labels = label_encoder.fit_transform(labels)
        labels = tf.keras.utils.to_categorical(labels, num_classes=self.config.classes)
        return labels

    def load_model(self):
        # Load the CNN-LSTM model prepared in Notebook 2
        self.model = tf.keras.models.load_model(self.config.cnn_lstm_model_path)

    def train(self):
        # Load and process the datasets
        train_data, train_labels = self.load_or_process_dataset(self.config.train_csv, 'train', self.config.max_seq_length)
        validate_data, validate_labels = self.load_or_process_dataset(self.config.validate_csv, 'validate', self.config.max_seq_length)

        # Encode the labels to numeric values
        train_labels = self.encode_labels(train_labels)
        validate_labels = self.encode_labels(validate_labels)

        # Load the model
        self.load_model()

        # Compile the model with the desired optimizer and loss function
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.config.learning_rate),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Train the model
        history = self.model.fit(
            train_data, train_labels,
            validation_data=(validate_data, validate_labels),
            epochs=self.config.params_epochs,
            batch_size=self.config.params_batch_size
        )

        # Save the trained model
        self.model.save(self.config.trained_model_path)
        print(f"Model saved successfully at {self.config.trained_model_path}")

        return history


In [16]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.train()

except Exception as e:
    raise e


[2024-08-29 00:07:20,836: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-29 00:07:20,850: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-29 00:07:20,851: INFO: common: created directory at: artifacts]
Loading existing train features and labels...
Loading existing validate features and labels...
[2024-08-29 00:07:36,862: WARNING: load: No training configuration found in save file, so the model was *not* compiled. Compile it manually.]
19/19 [==============================] - 3339s 176s/step - loss: 7.2259 - accuracy: 0.0400 - val_loss: 13.0295 - val_accuracy: 0.0400
[2024-08-29 01:03:42,336: WARNING: save: Found untraced functions such as _update_step_xla, lstm_cell_5_layer_call_fn, lstm_cell_5_layer_call_and_return_conditional_losses, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 56). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: artifacts\training\c